In [1]:
import os
import numpy as np
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler #用于数据归一化处理
from scipy.stats import ttest_ind, levene  # T 检验 方差齐性检验
from sklearn.linear_model import LassoCV 
from sklearn.utils import shuffle  # 数据混序
from sklearn.preprocessing import StandardScaler 

In [ ]:
!pip install pandas

In [2]:
os.getcwd()

'/home/xyy/1-jupyter/code'

In [15]:
dir="C:\\Users\\24993\\Desktop\\jupyter\\data\\3d_MRI_1024dim.xlsx"
meta_dir="C:\\Users\\24993\\Desktop\\jupyter\\data\\label.xlsx"

In [18]:
features=pd.read_excel(dir)
meta=pd.read_excel(meta_dir)

In [19]:
features=features.set_index("MRI")
meta=meta.set_index("MRI")
label=meta["pCR"]

In [20]:
pCR=label[label==1]
RD=label[label==0]
pCR=pCR.to_frame()
RD=RD.to_frame()

In [21]:
feature_pCR=pd.DataFrame(features,index=pCR.index)
feature_RD=pd.DataFrame(features,index=RD.index)

In [22]:
#ttest
counts=0
columns_index=[]
for column_name in feature_pCR.columns:
    if levene(feature_pCR[column_name],feature_RD[column_name])[1]>0.05:
        if ttest_ind(feature_pCR[column_name],feature_RD[column_name],equal_var=True)[1]<0.05:
            columns_index.append(column_name)
    else:
        if ttest_ind(feature_pCR[column_name],feature_RD[column_name],equal_var=False)[1]<0.05:
            columns_index.append(column_name)
print("筛选后的特征个数：{}个".format(len(columns_index)))

筛选后的特征个数：11个


In [23]:
data=features[columns_index]

In [24]:
data1=pd.DataFrame(data)
data1.index=data.index
data1.columns=data.columns

In [55]:
from sklearn import preprocessing   
 
min_max_scaler = preprocessing.MinMaxScaler()  
 
X_minMax = min_max_scaler.fit_transform(data1)  

In [61]:
data2=pd.DataFrame(X_minMax)
data2.index=data.index
data2.columns=data.columns

In [63]:
data2.to_excel("C:\\Users\\24993\\Desktop\\jupyter\\data\\grey60_ttest.xlsx")

In [52]:
X=data1
y=label

In [53]:
#boruta
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from boruta import BorutaPy

rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
boruta_model = BorutaPy(
        estimator=rf,
        n_estimators='auto',
        max_iter=100  # number of trials to perform
    )

# 模型训练
boruta_model.fit(X, y)
# 输出结果
boruta_model.support_    #筛选后的特征，False代表滤除掉此特征
boruta_model.ranking_   #1为选定的特征，2为暂定特征
green_area = X.columns[boruta_model.support_].to_list()
blue_area = X.columns[boruta_model.support_weak_].to_list() 
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [141]:
data_boruata=data1[green_area]

In [142]:
data_boruata.to_excel("/home/linjiatai/Research/Data/MRI/protein/protein_boruata.xlsx")

In [35]:
lasso_x=data1
lasso_y=meta["pCR"]

In [36]:
alphas = np.logspace(-4,1,50) 
# alphas 实际上是 λ 值，常量，通过模型优化选择，但可以给定范围，10e-4 到 1 范围内，等分 50 份，以 log 为间隔（以 10 为底，以等差数列中的每个值为指数），取 50 个值。

model_lassoCV = LassoCV(alphas = alphas, cv=5,max_iter = 100000).fit(lasso_x,lasso_y)

In [37]:
coef = pd.Series(model_lassoCV.coef_, index = lasso_x.columns)
print(model_lassoCV.alpha_)  # 选出的最优 alpha 值 0.00040949150623804275
print('%s %d'%('Lasso picked', sum(coef != 0))) # 系数不等于 0 的特殊个数 Lasso picked 21 
print(coef[coef != 0]) 

0.18420699693267165
Lasso picked 6
S100A12    0.000880
CLC        0.004679
FADS2      0.006220
PRG3       0.004596
PRG2       0.005007
TP53       0.014679
dtype: float64


In [40]:
coef_index=coef[coef != 0].index

In [41]:
lasso_result=lasso_x[coef_index]

In [64]:
X=data2

In [65]:
from sklearn.model_selection import train_test_split
#导入你要分的数据特征X，标签Y
X_train, X_test,y_train, y_test = train_test_split(X,label,test_size=0.2)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,roc_auc_score  
svm = SVC(kernel='poly',probability=True) # degree=3, random_state=42 

svm.fit(X_train, y_train)  

# 在测试集上进行预测  

y_pred = svm.predict(X_test)  
y_pred_proba = svm.predict_proba(X_test)[:, 1]
auc=roc_auc_score(y_test, y_pred_proba)

# 计算并打印准确率  

accuracy = accuracy_score(y_test, y_pred)  

print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC",auc)

Accuracy: 67.86%
AUC 0.7755102040816326
